In [67]:
import os
import gc
import pyperclip as clip
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")
from time import sleep
import sistemas
import sei
import functions
from page import *
import selenium.webdriver as webdriver

% reload_ext autoreload
% autoreload 2

In [68]:
#PATH = os.path.abspath(r"S:\PUBLICO\Outorga 3º Andar\Provas de Radioamador\2018.07.25\Corrigidos")
PATH = os.path.abspath(r"C:\Users\rsilva\Desktop\SEI")
print(PATH)
#SCAN = os.path.abspath(r"S:\PUBLICO\Outorga 3º Andar\Provas de Radioamador\2018.07.25\scaneados")

C:\Users\rsilva\Desktop\SEI


In [ ]:
keys = ('Nome/Razão Social', 'Logradouro', 'Número', 'Complemento', 'Bairro', 'Cep', 'Município', 'UF', 'Número Fistel')

In [ ]:
def string_endereço(dados):
    
    d = {}
    
    s = 'A(o)<br><br>' 
    
    s += dados["Nome/Razão Social"].title()
    
    s += '<br>' + dados["Logradouro"].title() + ", " + dados["Número"] + " " 
    
    s += dados["Complemento"].title() + " " 
    
    s += dados["Bairro"].title() + '<br>' 
    
    s += "CEP: " + dados["Cep"] + " - " + dados["Município"].title() + " - " + dados["UF"] 
            
    s += "<br><br>" + "<b>FISTEL: " + dados["Número Fistel"] + "</b>"
    
    d["A"] = s
    
    return d

In [56]:
cpf = '25578881840'
proc = r'53504.009518/2018-31'

In [3]:
driver = webdriver.Ie()
#scpx = sistemas.Scpx(driver)


# Scpx - Manipulação de Cadastro

## Consulta

In [ ]:
scpx.consulta(cpf)

## Serviço

In [ ]:
scpx.servico_incluir(cpf, num_processo=proc, silent=True)

In [ ]:
#scpx.servico_excluir(cpf, documento="2724547")
scpx.imprime_consulta(cpf)

## Estação

In [ ]:
scpx.prorrogar_rf(cpf)

In [ ]:
scpx.prorrogar_estacao(cpf)

In [ ]:
scpx.incluir_estacao(cpf, 'Móvel', 'PX2A1957', sequencial='001')

In [ ]:
scpx.incluir_estacao(cpf, 'Fixa', 'PX2Y3769', sequencial='002')

In [ ]:
scpx.movimento_transferir(cpf, 'B', 'E', proc=proc)

In [ ]:
scpx.movimento_transferir(cpf, 'A', 'G', proc=proc)

In [ ]:
scpx.movimento_cancelar(cpf)

In [ ]:
scpx.licenciar_estacao(cpf, silent=True)

In [ ]:
scpx.licenciar_estacao(cpf, ppdess=False, silent=True)

## Licença

In [ ]:
scpx.imprimir_licenca(cpf)

# Sigec

In [87]:
sigec = sistemas.Sigec(driver)

In [ ]:
sigec.consulta_geral(cpf)

In [ ]:
dados.values()

# Sec

In [106]:
import pandas as pd

df = pd.read_excel("files/registros_atualizados.xlsx", dtype=str, na_values='nan').fillna("")

In [107]:
df['Ano do Óbito'] = df["Ano do Óbito"].str.replace('nan', "")

df['Complemento'] = df["Complemento"].str.replace('nan', "")

df["Data de Nascimento"] = df["Data de Nascimento"].str.replace("-", '')

df["Complemento"] = df["Complemento"].str.replace('nan', "")

df['Cidade'] = df["Cidade"].apply(lambda x: str(x).split("/")[0])

In [109]:
df = df.iloc[:8335]
df["Erros"] = ""

In [111]:
total = df.shape[0]

In [112]:
driver = webdriver.Firefox()
sec = sistemas.Sec(driver, 'rsilva', "Savorthemom3nts")

In [115]:
for i in range(10,100):
    
    try:
        sec.atualiza_cadastro(df.iloc[i])
        
    except:
        
        df.loc[i, "Erros"] = True
        
        next
        
        gc.collect()   
    

# SEI - Manipulação de Processo

## Autenticação

In [47]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [48]:
sei_ = sei.login_sei(browser, usr, pwd)

In [49]:
sei_ = sei.Sei(sei_)

In [50]:
sei_.itera_processos()

In [57]:
p = sei_.go_to_processo(proc)

## Doc Externo

In [58]:
files = sorted(os.listdir(PATH))

In [59]:
file = os.path.join(PATH, files[-1])

In [60]:
p.incluir_doc_externo('AR', file, arvore="por E-mail")

In [ ]:
dados = scpx.extrai_cadastro(cpf)

In [31]:
p = sei_.go_to_processo(proc)

In [ ]:
p.incluir_oficio('RC_OF_Sem_Contexto_Licença', string_endereço(dados))

In [42]:
p = sei_.go_to_processo(proc)

In [ ]:
p.incluir_informe()

In [ ]:
for prova, gabarito in zip(files, scaneados):

    nome = prova.split(".")[0]
    
    p.incluir_doc_externo("Prova", os.path.join(SCAN, gabarito), arvore=nome)
    
    sleep(5)
    

## Controles

In [ ]:
p.edita_marcador(tipo="RC - Aguardando AR/Resposta", content='Até 20/08/2018')

In [ ]:
p.edita_postit("Analisado - Controlar pagamento e Emitir Licença", True)

In [61]:
p.edita_marcador()

In [44]:
p.edita_postit()

In [62]:
p.excluir_acomp_especial()

In [63]:
p.concluir_processo()

# Testes

In [ ]:
tipo = "Outorga: Rádio do Cidadão"
atribuicao = 'rodrigoca'

In [ ]:
processos = [p for p in sei_.get_processos().values() if p['atribuicao'] == 'rodrigoca']

In [ ]:
mm = [p for p in processos if p['tipo'] == tipo]

In [ ]:
for p in mm:
    print(p.get('marcador'))

In [ ]:
concluir = list()

In [ ]:
for processo in processos:
    
    if 'Licença' in str(processo.get('marcador')):
        
        p = sei_.go_to_processo(processo['numero'])
        
        i = input()
        
        if i == '1': 
            
            concluir.append(processo)
            
            
        gc.collect()

In [ ]:
for processo in concluir:
    
    p = sei_.go_to_processo(processo['numero'])
    
    
    sleep(2)
    
    try:    
        p.edita_marcador()    
    
        p.edita_postit()
    
        p.excluir_acomp_especial()
        
        p.concluir_processo() 
            
    except:
            
            next

In [ ]:
#dados = scpx.extrai_cadastro(dado['interessado'], tipo_id='id_nome', timeout=5)


In [ ]:
for num, dado in tupla[30:]:
        
    p = sei_.go_to_processo(num)
    
    i = input()
    
    if i =='y':
        
        try:
            
            dados = scpx.extrai_cadastro(dado['interessado'], tipo_id='id_nome', timeout=5)
                               
            p.incluir_oficio('RC_OF_Sem_Contexto_Licença', string_endereço(dados))
            
            p.edita_marcador(tipo='RC - Aguardando AR/Resposta', content='Até 31/07/2018')        

            
        except:
            
            next            
            
    if i == 'c':
        
        p.edita_marcador()
        p.edita_postit()
        p.excluir_acomp_especial()
        p.concluir_processo() 

## Outros

In [ ]:
from bs4 import BeautifulSoup as soup
html = soup(scpx.driver.page_source, 'lxml')

In [ ]:
dict_ = {}
for tr in html.find_all('tr'):
    for td in tr.find_all('td', string=True):
        key = td.text.strip(" :")
        value = td.find_next_sibling('td')            
        if value and key not in dict_ and value.text.strip() != "":
            dict_[key] = value.text.strip()
            print("{0} {1}".format(key, value.text.strip())) 


In [ ]:
dict_ = {}
for tr in html.find_all('tr'):
    for label in tr.find_all('label', string=True):
        print("{0} {1}".format(label.attrs.get('id'), label.text.strip()))                           

In [ ]:
from bs4 import NavigableString
def next_is_string(tag):
    return isinstance(tag.next_element, NavigableString)             

# Boleto

In [ ]:
boleto = sistemas.Boleto(driver)

In [ ]:
fistel = '80113639805'
cpf = "31250732867"

In [ ]:
boleto.imprime_boleto(fistel, tipo_id='id_fistel')

In [ ]:
gc.collect()